In [ ]:
%load_ext autoreload
%autoreload 2

from feature_extraction import Method
from readers import GetProcessedInputData
from ipynb.fs.full.accuracy_metrics import CalculateAccuracy


feature_extraction_methods = [Method.BLACK_PIXEL_RATIO]
transcriptions_as_list, keywords_to_search, train_words, test_words = GetProcessedInputData(feature_extraction_methods)

In [ ]:
# TODO: For testing only. Remove the entire cell later

# all_train_words = train_words
# all_test_words = test_words

train_words = train_words[:7]
test_words = test_words[:7]

In [ ]:
import numpy as np
from dtaidistance import dtw
from compute_distances import ComputeDistances


distances = ComputeDistances(train_words, test_words)

In [ ]:
from keyword_spotting import FindClosestKnownWord, SpotKeyword


# For each test_word, find the train_word that is closest to it.
for test_word in test_words:
  test_word.closest_train_word, test_word.distance_to_closest_train_word = \
    FindClosestKnownWord(test_word, train_words, distances)
    

# Search keywords and save spotted keywords in the list.
outputData = []
for keyword_to_search in keywords_to_search:
  spotted_test_keywords = SpotKeyword(keyword_to_search, test_words)
  if len(spotted_test_keywords):
    print(keyword_to_search)
    print(spotted_test_keywords)
    spottedKeyword = []
    for keyword in spotted_test_keywords:
      spottedKeyword.append(keyword.id)
      spottedKeyword.append(keyword.distance_to_closest_train_word)
      print(keyword.distance_to_closest_train_word)
      keyword.image.show()
    outputData.append([keyword_to_search, *spottedKeyword])
    input()

In [ ]:
# TODO: write output file kws.csv (slide 24)
import csv
with open("kws.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(outputData)

In [ ]:
# Evaluation Metrics - In progress, not finished yet (There might be errors in the logic)
# convert transcriptions to dict - easier and faster access values later
from ipynb.fs.full.accuracy_metrics import CalculateAccuracy
groundTruth = dict()
for sub in transcriptions_as_list:
    groundTruth[sub[0]] = sub[1]

testWords = dict()
for word in test_words:
    testWords[word.id] = word.transcription

predictions = dict()
for item in outputData:
    predictions[item[0]] = item[1::2]

acc = CalculateAccuracy(keywords_to_search, testWords, predictions, groundTruth)


print(acc.CalculateRecall())
print(acc.CalculatePrecision())
